# Operações de Álgebra Linear e Cálculo

## 1. Operações de Álgebra Linear 

### 1.1. Somas e Reduções de Dimensionalidade ; Transmissão ("BroadCasting")

Vamos importar o tensorflow e criar uma matriz 

In [1]:
import tensorflow as tf
A=tf.reshape(tf.range(6),(2,3))
A

2024-04-03 17:37:55.950944: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 17:37:57.058707: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-03 17:37:58.330909: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-03 17:37:58.344678: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[0, 1, 2],
       [3, 4, 5]], dtype=int32)>

Observe que é um array bidimensional 2x3. Frequentemente precisamos somar as linhas ou colunas de matrizes (para fazer normalizações, por exemplo). 

In [2]:
tf.reduce_sum(A)

<tf.Tensor: shape=(), dtype=int32, numpy=15>

O ``reduce_sum``, sem parâmetros, soma todos os elementos da matriz A, e devolve um escalar (dimensão 0). 

In [3]:
tf.reduce_sum(A,axis=0)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([3, 5, 7], dtype=int32)>

Indicando-se a dimensão 0 (axis), apenas as linhas são colapsadas, isto é, temos um vetor de tamanho 3 (array monodimensional) constituído pela soma de cada coluna. Observe como a função ``reduce_sum`` a princípio diminui a dimensionalidade do objeto. A função ``reduce_mean`` tem comportamento semelhante, mas com a média (e não a soma) de elementos. 

Esta diminuição de dimensionalidade nem sempre é desejável. Digamos que calculemos a soma das linhas de A, para normalizá-las (fazer toda linha ter soma 1). Não podemos fazer simplesmente ``A/reduce_sum(A,axis=1)`` porque estaríamos dividindo uma matriz (dimensão 2) por um vetor (dimensão 1). A solução é usar a opção ``keepdims=True`` para que as dimensões de tamanho 1 não sejam colapsadas. Em outras palavras, para que esta soma de linhas seja representada em uma matriz 2x1 e não um vetor 2.

In [4]:
SomaLinhas=tf.reduce_sum(A,axis=1)
SomaLinhas2=tf.reduce_sum(A,axis=1,keepdims=True)
SomaLinhas,SomaLinhas2

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 3, 12], dtype=int32)>,
 <tf.Tensor: shape=(2, 1), dtype=int32, numpy=
 array([[ 3],
        [12]], dtype=int32)>)

In [5]:
A/SomaLinhas2

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[0.        , 0.33333333, 0.66666667],
       [0.25      , 0.33333333, 0.41666667]])>

Mas dissemos anteriormente que a divisão / é realizada elemento a elemento. A e SomaLinhas 2 têm dimensão 2, mas os tamanhos não são os mesmos (2x3 contra 2x1). Como a operação pôde ser realizada corretamente? A resposta está na importante característica de **Broadcasting** (transmissão) pressuposta nas operações matriciais no TensorFlow. Antes de realizar a operação, as dimensões de tamanho 1 são replicadas para que tenhamos tamanhos compatíveis. Em outras palavras, interpreta-se que o que se deseja é fazer 3 cópias (em colunas) da matriz 2x1 para que se tenha uma matriz 2x3. Observe que a operação é exatamente o que queríamos: a soma das linhas é 1. 

"Broadcasting" será muito útil na manipulação de grandes tabelas de dados em Aprendizado de Máquina. 

### 1.2. Produto interno, produto matriz-vetor e matriz-matriz

A função ``tf.tensordot`` realiza o produto interno de vetores. 

In [6]:
x,y=tf.range(3),tf.range(3)
x,y,tf.tensordot(x,y,axes=1)

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([0, 1, 2], dtype=int32)>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([0, 1, 2], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=5>)

Já a função ``tf.matmul``realiza o produto matriz x vetor. 

In [7]:
W=tf.random.normal(shape=(3,3))
W

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-0.5663028 , -0.75543094,  0.44377095],
       [-1.4696789 , -0.12964225, -1.2041924 ],
       [-0.8877599 , -0.5225164 ,  1.6861261 ]], dtype=float32)>

In [8]:
Y=tf.random.normal(shape=(3,3))
tf.matmul(W,Y)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-0.14805588,  1.0885344 ,  0.4512012 ],
       [-1.6853718 , -0.34593436,  1.0268487 ],
       [-0.25259694,  1.6683149 ,  0.48578602]], dtype=float32)>

### 1.3. Normas

A função ``tf.norm`` calcula a norma L2 (ou euclidiana) de um vetor. 

In [9]:
A=tf.constant([3.0,4.0])
tf.norm(A)

<tf.Tensor: shape=(), dtype=float32, numpy=5.0>

O ".0" ao final faz com que A seja um vetor de ``float32``. A função ``tf.norm``não aceita inteiros como argumentos. É possível contornar isso com a função ``tf.cast`` 

In [10]:
A = tf.constant([3, 4])
tf.norm(tf.cast(A, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=5.0>

Se aplicada a uma matriz, ``tf.norm`` apresenta a norma de Frobenius da matriz (raiz quadrada da soma dos quadrados de todos os elementos).

In [11]:
A=tf.ones((3,3))
tf.norm(A)

<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

Observe que a função ``tf.ones`` devolve um vetor de ``float32``, não sendo necessário o "casting".

## 2. Cálculo: Diferenciação automática

Na Figura abaixo, temos um neurônio artificial com entradas $\vec{x}$, pesos sinápticos $\vec{w}$ e saída $y$.

![Neuronio Artificial](./Neuronio.png)

Diferente do modelo que vimos aula passada, este neurônio não computa apenas o produto interno entre $\vec{w}$ e $\vec{x}$ (ou dito de outra forma, faz a soma das entradas $\vec{x}$ ponderada pelos pesos $\vec{w}$). Ele também passa a soma resultante por uma função não-linear, conhecida como logística ou sigmoide: 
\begin{equation*}
y=\frac{1}{1+\exp(-s)}
\end{equation*},
onde $s$ é a soma.

Frequentemente queremos saber qual a derivada de $y$ com relação a cada peso $w_{i}$. Isto será usado para modificar $w$ de acordo com os erros que a rede neural comete. 

TensorFlow permite o cálculo automático das derivadas de variáveis computadas. 

Primeiro, vamos simular um passo do cálculo da saída do neurônio. 

In [12]:
x=tf.Variable(tf.constant([1.0,-2.0,3.0]))
w=tf.Variable(tf.constant([0.2,-0.2,1]))

Encontramos pela primeira vez a função ``tf.Variable``. Esta é uma ferramenta de economia de memória. Python usa gerenciamento dinâmico de memória, o que significa que ele vai alocando memória ao longo da execução. Isto pode ser útil no gerenciamento de memória (podemos ir descartando a memória que não será mais usada), mas se usado sem cuidado pode ter o efeito contrário. Quando fazemos ``Y=X+Y`` alocamos memória para `X+Y` mesmo sendo o objetivo apenas atualizar `Y`, por exemplo. 

Para garantir que algo que será calculado várias vezes (como os parâmetros de uma rede neural) sejam alocados em uma mesma porção de memória, existe a função ``tf.Variable``. Ela recebe o valor inicial da variável e, futuramente, o valor pode ser mudado sem alocar nova memória. 

Resolvida a questão da alocação, podemos calcular o valor da saída. Mas queremos que este cálculo seja usado para que o gradiente da saída em relação aos pesos $\vec{w}$ seja calculado automaticamente. Para isto, colocamos o código onde a conta é feita sob a declaração ``with tf.GradientTape() as t:``. A declaração ``with`` em Python define um encapsulamento e um contexto para o código. É comum para arquivos (veja o notebook/caderno anterior), que podem ser fechados após a leitura, ou justamente para declarar que as operações devem ser memorizadas para que se compute o gradiente, como aqui. 

In [13]:
with tf.GradientTape() as t:
    y=tf.sigmoid(tf.tensordot(x,w,axes=1))
y

<tf.Tensor: shape=(), dtype=float32, numpy=0.973403>

Observe que a computação está correta. O produto interno nos fornece $1\times 0.2+(-2)\times(-0.2)+3\times 1 = 3,6$. Verifique que a função logística, definida acima, vale $0,97$ para $s=3,6$. 

Agora vamos ao gradiente. 

In [14]:
dydw = t.gradient(y, w)
dydw

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.02588962, -0.05177924,  0.07766887], dtype=float32)>

De novo, verifique que a computação está correta. Para calcular a derivada parcial de $y$ com relação a cada $w_{i}$ aplique a regra da cadeia. $$\frac{\partial y}{\partial w_{i}}=\frac{dy}{ds}\cdot\frac{\partial s}{w_{i}}$$

Compute a derivada de $y$ com relação a $s$, verificando que vale $s(1-s)$. O outro fator, já que $s$ é simplesmente a soma ponderada, vale $x_{i}$. Multiplique os fatores e verifique o resultado. 